In [82]:
cc_keys = '''906
907
908
910
900A
974A
974AB'''.split('\n')

In [83]:
print(tuple(set(sorted(cc_keys))))

('908', '974A', '907', '974AB', '910', '906', '900A')


In [11]:
l = ['boo']
if len(l) <= 1:
    l2 = f'({l[0]})'
l2

'(boo)'

In [37]:
from itertools import chain
l = ('7722.001')
l2 = ('3256', '3588', '6937', '7722')
# tuple(chain((a for a in l), (b for b in l2)))
tuple(list(l,) + list(l2,))

('7', '7', '2', '2', '.', '0', '0', '1', '3256', '3588', '6937', '7722')

In [16]:
t = ('3256', '7618')
len(t)

2

In [2]:
l = ('(7722.001)')
l2 = (['(7790.001)'])
print(tuple(l), tuple(l2))

('(', '7', '7', '2', '2', '.', '0', '0', '1', ')') ('(7790.001)',)


In [16]:
import re

pattern = r'[\(,\)]+'
string = '(7790.001)'
repl = ''

re.sub(r'[\(,\)]+', '', string)

'7790.001'

In [22]:
pattern = r'[\(,\',\)]'
string = "('7790')"
repl = ''
one_job = re.sub(pattern, repl, string)
one_job

'7790'

In [26]:
tuple([one_job] + list(('7790.002',)))

('7790', '7790.002')

In [1]:
import streamlit as st
from snowflake import connector
import pandas as pd
import numpy as np
import os
from os.path import join, dirname
from dotenv import load_dotenv
import re
from mitosheet.streamlit.v1 import spreadsheet

# get environment variables
dotenv_path = join(dirname('./app/streamlit_labor_lense'), '.env')
load_dotenv(dotenv_path)
SF_ACCOUNT = os.getenv('SF_ACCOUNT')
SF_USER = os.getenv('SF_USER')
SF_PASSWORD = os.getenv('SF_PASSWORD')
SF_ROLE = os.getenv('SF_ROLE')
SF_WAREHOUSE = os.getenv('SF_WAREHOUSE')
SF_DATABASE = os.getenv('SF_DATABASE')
SF_SCHEMA = os.getenv('SF_SCHEMA')

def load_data(query):
    cur = create_connection()
    df_data = cur.execute(query).fetch_pandas_all()
    return df_data

def create_connection():
    conn = connector.connect(
        user = SF_USER
        ,password = SF_PASSWORD
        ,account = SF_ACCOUNT
        ,warehouse = SF_WAREHOUSE
        ,database = SF_DATABASE
        ,schema = SF_SCHEMA
        ,role = SF_ROLE
    )
    cur = conn.cursor()
    cur.execute('use warehouse prod_ts_elt_warehouse;')
    return cur

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
SF_USER

'svc_prod_ts_labor_lense'

In [17]:
# all_job_numbers = "('7484', '7618', '7722', '7790', '7954', '7834', '7484.098', '7484.099', '7484.999', '7618.001', '7618.002', '7618.003', '7618.004', '7618.005', '7618.006', '7618.007', '7618.050', '7618.099', '7618.100', '7618.101', '7618.102', '7618.999', '7722.001', '7722.002', '7722.003', '7722.004', '7722.096', '7722.097', '7790.002', '7790.097', '7790.099', '7954.100', '7954.101', '7834.001', '7834.098')"
all_job_numbers = "('7618.001', '7790')" #, '7722', '7790', '7954', '7834', '7484.098', '7484.099', '7484.999', '7618.001', '7618.002', '7618.003', '7618.004', '7618.005', '7618.006', '7618.007', '7618.050', '7618.099', '7618.100', '7618.101', '7618.102', '7618.999', '7722.001', '7722.002', '7722.003', '7722.004', '7722.096', '7722.097', '7790.002', '7790.097', '7790.099', '7954.100', '7954.101', '7834.001', '7834.098')"
query = f'select * from prod_publish.global.labor_lense where contract in {all_job_numbers}'
df_data = load_data(query)

In [18]:
df_data = pd.DataFrame(df_data)
# df_data.MAIN_CONTRACT.unique() 

In [5]:
df_data.columns

Index(['MAIN_CONTRACT', 'CONTRACT', 'CRAFT', 'CLASS', 'ROLE_L1', 'ROLE_L2',
       'ROLE_L3', 'JOB_TITLE', 'JOB_CLASS', 'JOB_TRADE', 'REG_HOURS',
       'OVT_HOURS', 'DT_HOURS', 'OTHER_HOURS', 'TOTAL_HOURS', 'JOB_TOTAL_COST',
       'LINE_TOTAL_COST', 'LABOR_COST', 'PRICE_PER_HOUR', 'BURDEN_COST',
       'START_MONTH', 'END_MONTH', 'DURATION_MONTHS'],
      dtype='object')

In [54]:
df_craft = df_data.loc[df_data['ROLE_L1'] == 'Craft'].groupby(by=['MAIN_CONTRACT', 'ROLE_L2']).agg({'TOTAL_HOURS': 'sum', 'REG_HOURS': 'sum', 'OVT_HOURS': 'sum', 'DT_HOURS': 'sum', 'LABOR_COST': 'sum', 'BURDEN_COST': 'sum', 'LINE_TOTAL_COST': 'sum'})
df_craft = df_craft.reset_index()
df_craft['LABOR_COST_PER_HOUR'] = df_craft.LABOR_COST / df_craft.TOTAL_HOURS
df_craft['BURDEN_COST_PRCT'] = df_craft.BURDEN_COST / df_craft.LABOR_COST
df_craft['REG_HRS_PRCT'] = df_craft.REG_HOURS / df_craft.TOTAL_HOURS
df_craft['OVT_HRS_PRCT'] = df_craft.OVT_HOURS / df_craft.TOTAL_HOURS
df_craft['DT_HRS_PRCT'] = df_craft.DT_HOURS / df_craft.TOTAL_HOURS
df_craft = df_craft.pivot(index='ROLE_L2', columns='MAIN_CONTRACT', values=['TOTAL_HOURS', 'LABOR_COST', 'LABOR_COST_PER_HOUR', 'BURDEN_COST', 'BURDEN_COST_PRCT', 'REG_HRS_PRCT', 'OVT_HRS_PRCT', 'DT_HRS_PRCT', 'LINE_TOTAL_COST'])
df_craft = df_craft.reset_index(names='ROLE')
df_craft['ROLE'] = df_craft['ROLE'].apply(lambda x: f'{x} Composite Crew')
for _ in df_craft['LABOR_COST'].columns:
    df_craft['TRADE_PRCT_OF_CRAFT_SUBTOT', _] = df_craft['LABOR_COST', _] / df_craft['LABOR_COST', _].sum()

df_subtotals = df_data.loc[df_data.TOTAL_HOURS != 0].groupby(by=['MAIN_CONTRACT', 'ROLE_L1']).agg({'TOTAL_HOURS': 'sum', 'REG_HOURS': 'sum', 'OVT_HOURS': 'sum', 'DT_HOURS': 'sum', 'LABOR_COST': 'sum', 'BURDEN_COST': 'sum', 'LINE_TOTAL_COST': 'sum'})
df_subtotals = df_subtotals.reset_index()
df_subtotals['LABOR_COST_PER_HOUR'] = df_subtotals.LABOR_COST / df_subtotals.TOTAL_HOURS
df_subtotals['BURDEN_COST_PRCT'] = df_subtotals.BURDEN_COST / df_subtotals.LABOR_COST
df_subtotals['REG_HRS_PRCT'] = df_subtotals.REG_HOURS / df_subtotals.TOTAL_HOURS
df_subtotals['OVT_HRS_PRCT'] = df_subtotals.OVT_HOURS / df_subtotals.TOTAL_HOURS
df_subtotals['DT_HRS_PRCT'] = df_subtotals.DT_HOURS / df_subtotals.TOTAL_HOURS
df_subtotals = df_subtotals.pivot(index='ROLE_L1', columns='MAIN_CONTRACT', values=['TOTAL_HOURS', 'LABOR_COST', 'LABOR_COST_PER_HOUR', 'BURDEN_COST', 'BURDEN_COST_PRCT', 'REG_HRS_PRCT', 'OVT_HRS_PRCT', 'DT_HRS_PRCT', 'LINE_TOTAL_COST'])
df_subtotals = df_subtotals.reset_index(names='ROLE')
df_subtotals['ROLE'] = df_subtotals['ROLE'].apply(lambda x: f'{x} Subtotal')

df_total = df_data.groupby(by=['MAIN_CONTRACT']).agg({'TOTAL_HOURS': 'sum', 'REG_HOURS': 'sum', 'OVT_HOURS': 'sum', 'DT_HOURS': 'sum', 'LABOR_COST': 'sum', 'BURDEN_COST': 'sum', 'LINE_TOTAL_COST': 'sum'})
df_total = df_total.reset_index()
df_total['ROLE'] = 'Grand Total'
df_total['LABOR_COST_PER_HOUR'] = df_total.LABOR_COST / df_total.TOTAL_HOURS
df_total['BURDEN_COST_PRCT'] = df_total.BURDEN_COST / df_total.LABOR_COST
df_total['REG_HRS_PRCT'] = df_total.REG_HOURS / df_total.TOTAL_HOURS
df_total['OVT_HRS_PRCT'] = df_total.OVT_HOURS / df_total.TOTAL_HOURS
df_total['DT_HRS_PRCT'] = df_total.DT_HOURS / df_total.TOTAL_HOURS
df_total = df_total.pivot(index='ROLE', columns='MAIN_CONTRACT', values=['TOTAL_HOURS', 'LABOR_COST', 'LABOR_COST_PER_HOUR', 'BURDEN_COST', 'BURDEN_COST_PRCT', 'REG_HRS_PRCT', 'OVT_HRS_PRCT', 'DT_HRS_PRCT', 'LINE_TOTAL_COST'])
df_total = df_total.reset_index(names='ROLE')
df = pd.concat([df_craft, df_subtotals, df_total])
df.columns.name = 'HOURS'
df = df.reset_index(drop=True).set_index('ROLE')
for _ in df['LABOR_COST'].columns:
    labor_total = df['LABOR_COST', _].loc[~df.index.get_level_values(0).isin(['Craft Subtotal', 'Grand Total'])].sum()
    df['TRADE_PRCT_OF_LABOR_TOTAL', _] = df['LABOR_COST', _] / labor_total
for _ in df['LABOR_COST'].columns:
    grand_total = df['LINE_TOTAL_COST', _].loc[df.index.get_level_values(0).isin(['Grand Total'])].sum()
    df['TRADE_PRCT_OF_JOB_TOTAL', _] = df['LABOR_COST', _] / grand_total

In [58]:
df_ovt = df[['OVT_HRS_PRCT']]
df_ovt.columns = df_ovt.columns.droplevel()
ovt_index = [_*2+1 for _ in range(len(df_ovt.index))]
df_ovt = df_ovt.reset_index()
df_ovt.index = ovt_index
df_ovt['ROLE'] = df_ovt['ROLE'].apply(lambda x: f'{x} OT')
df_dt = df[['DT_HRS_PRCT']]
df_dt.columns = df_dt.columns.droplevel()
dt_index = [_*2+2 if _ > 0 else 2 for _ in range(len(df_dt.index))]
df_dt = df_dt.reset_index()
df_dt.index = dt_index
df_dt['ROLE'] = df_dt['ROLE'].apply(lambda x: f'{x} DT')
df_ovt_dt = pd.concat([df_ovt, df_dt])
df_ovt_dt = df_ovt_dt.sort_index().set_index('ROLE')
df_ovt_dt

MAIN_CONTRACT,7618,7790
ROLE,,
Carpenter Composite Crew OT,0.06,0.09
Carpenter Composite Crew DT,0.00,0.00
Cement Mason Composite Crew OT,0.07,0.12
Cement Mason Composite Crew DT,0.00,0.01
Ironworker Composite Crew OT,0.06,0.02
Ironworker Composite Crew DT,0.00,0.00
Laborer Composite Crew OT,0.06,0.11
Laborer Composite Crew DT,0.00,0.01
Operator Composite Crew OT,0.13,0.16


In [11]:
df_gt = df_data.groupby(by=['MAIN_CONTRACT']).agg({'JOB_TOTAL_COST': 'max'}).style.format('${:,.2f}')
df_gt

,JOB_TOTAL_COST
MAIN_CONTRACT,
7484,"$331,805,361.61"
7618,"$106,534,981.50"


In [22]:
# df_data
df_craft_hours = df_data.loc[df_data['ROLE_L1'] == 'Craft'].groupby(by=['MAIN_CONTRACT', 'ROLE_L2']).agg({'TOTAL_HOURS': 'sum', 'LABOR_COST': 'sum', 'BURDEN_COST': 'sum', 'JOB_TOTAL_COST': 'max'})
df_craft_hours = df_craft_hours.reset_index()
df_craft_hours['LABOR_COST_PER_HOUR'] = df_craft_hours.LABOR_COST / df_craft_hours.TOTAL_HOURS
df_craft_hours['BURDEN_COST_PRCT'] = df_craft_hours.BURDEN_COST / df_craft_hours.LABOR_COST
df_craft_hours = df_craft_hours.pivot(index='ROLE_L2', columns='MAIN_CONTRACT', values=['TOTAL_HOURS', 'LABOR_COST', 'LABOR_COST_PER_HOUR', 'BURDEN_COST', 'BURDEN_COST_PRCT', 'JOB_TOTAL_COST'])
df_craft_hours = df_craft_hours.reset_index(names='ROLE')
df_craft_hours['ROLE'] = df_craft_hours['ROLE'].apply(lambda x: f'{x} Composite Crew')
for _ in df_craft_hours['LABOR_COST'].columns:
    df_craft_hours['TRADE_PRCT_OF_CRAFT_SUBTOT', _] = df_craft_hours['LABOR_COST', _] / df_craft_hours['LABOR_COST', _].sum()
df_craft_hours

,ROLE,TOTAL_HOURS,LABOR_COST,LABOR_COST_PER_HOUR,BURDEN_COST,BURDEN_COST_PRCT,JOB_TOTAL_COST,TRADE_PRCT_OF_CRAFT_SUBTOT
MAIN_CONTRACT,,7484,7484,7484,7484,7484,7484,7484
0,Carpenter Composite Crew,"207,480.50","10,040,247.35",48.39,"5,352,725.55",0.53,"337,145,526.67",0.56
1,Cement Mason Composite Crew,"33,024.00","1,571,150.63",47.58,"882,291.94",0.56,"337,145,526.67",0.09
2,Ironworker Composite Crew,"28,849.54","1,396,030.32",48.39,"1,018,232.80",0.73,"337,145,526.67",0.08
3,Laborer Composite Crew,"83,439.50","3,501,890.73",41.97,"1,664,797.77",0.48,"337,145,526.67",0.19
4,Operator Composite Crew,"29,639.36","1,495,963.02",50.47,"891,061.87",0.60,"337,145,526.67",0.08


In [23]:
for _ in df_craft_hours['LABOR_COST'].columns:
    df_craft_hours['TRADE_PRCT_OF_CRAFT_SUBTOT', _] = df_craft_hours['LABOR_COST', _] / df_craft_hours['LABOR_COST', _].sum()
df_craft_hours

,ROLE,TOTAL_HOURS,LABOR_COST,LABOR_COST_PER_HOUR,BURDEN_COST,BURDEN_COST_PRCT,JOB_TOTAL_COST,TRADE_PRCT_OF_CRAFT_SUBTOT
MAIN_CONTRACT,,7484,7484,7484,7484,7484,7484,7484
0,Carpenter Composite Crew,"207,480.50","10,040,247.35",48.39,"5,352,725.55",0.53,"337,145,526.67",0.56
1,Cement Mason Composite Crew,"33,024.00","1,571,150.63",47.58,"882,291.94",0.56,"337,145,526.67",0.09
2,Ironworker Composite Crew,"28,849.54","1,396,030.32",48.39,"1,018,232.80",0.73,"337,145,526.67",0.08
3,Laborer Composite Crew,"83,439.50","3,501,890.73",41.97,"1,664,797.77",0.48,"337,145,526.67",0.19
4,Operator Composite Crew,"29,639.36","1,495,963.02",50.47,"891,061.87",0.60,"337,145,526.67",0.08


In [24]:
df_supervision_hours = df_data.loc[df_data.TOTAL_HOURS != 0].groupby(by=['MAIN_CONTRACT', 'ROLE_L1']).agg({'TOTAL_HOURS': 'sum', 'LABOR_COST': 'sum', 'BURDEN_COST': 'sum', 'JOB_TOTAL_COST': 'max'})
df_supervision_hours = df_supervision_hours.reset_index()
df_supervision_hours['LABOR_COST_PER_HOUR'] = df_supervision_hours.LABOR_COST / df_supervision_hours.TOTAL_HOURS
df_supervision_hours['BURDEN_COST_PRCT'] = df_supervision_hours.BURDEN_COST / df_supervision_hours.LABOR_COST
df_supervision_hours = df_supervision_hours.pivot(index='ROLE_L1', columns='MAIN_CONTRACT', values=['TOTAL_HOURS', 'LABOR_COST', 'LABOR_COST_PER_HOUR', 'BURDEN_COST', 'BURDEN_COST_PRCT', 'JOB_TOTAL_COST'])
df_supervision_hours = df_supervision_hours.reset_index(names='ROLE')
df_supervision_hours['ROLE'] = df_supervision_hours['ROLE'].apply(lambda x: f'{x} Subtotal')
df_supervision_hours

,ROLE,TOTAL_HOURS,LABOR_COST,LABOR_COST_PER_HOUR,BURDEN_COST,BURDEN_COST_PRCT,JOB_TOTAL_COST
MAIN_CONTRACT,,7484,7484,7484,7484,7484,7484
0,Craft Subtotal,"382,432.90","18,005,072.05",47.08,"9,809,082.54",0.54,"337,145,526.67"
1,Supervision Subtotal,"149,055.58","8,409,551.87",56.42,"2,976,047.94",0.35,"337,145,526.67"
2,Undefined Subtotal,-4.00,139.60,-34.90,13.61,0.10,"337,145,526.67"


In [10]:
# def make_pretty(styler):
#     styler.float_format = '{:,.2f}'.format
#     # styler.set_caption("Weather Conditions")
#     # styler.format(rain_condition)
#     # styler.format_index(lambda v: v.strftime("%A"))
#     # styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
#     return styler
df_burd = df_supervision_hours['BURDEN_COST_PRCT'] #.iloc[:,1:2]
df_burd = df_burd.style.format('{:.2f}%')
df_burd

MAIN_CONTRACT,7618,7790
0,0.55%,0.54%
1,0.32%,0.35%


df_supervision_hours.set_index('ROLE')

In [11]:
# def to_usd(x):
#         return '${:,.2f}'.format
# # df_supervision_hours[['TOTAL_HOURS']].apply(to_usd)
# # df_supervision_hours['TOTAL_HOURS'] = df_supervision_hours['TOTAL_HOURS'].apply(lambda x: '{:,.2f}'.format)
# # # df_supervision_hours['TOTAL_HOURS'] = df_supervision_hours['TOTAL_HOURS'].apply(to_usd)
# for _ in df_supervision_hours['BURDEN_COST_PRCT'].columns.values:
#         df_supervision_hours['BURDEN_COST_PRCT'][_] = df_supervision_hours['BURDEN_COST_PRCT'][_].style.format('{:.2f}%')
# # df_supervision_hours['TOTAL_HOURS'] = df_supervision_hours['TOTAL_HOURS'].apply('${:,.2f}'.format)
# # df_supervision_hours['TOTAL_HOURS'] = df_supervision_hours['TOTAL_HOURS'].apply('${:,.2f}'.format)
# # df_supervision_hours['TOTAL_HOURS'] = df_supervision_hours['TOTAL_HOURS'].apply('${:,.2f}'.format)
# df_supervision_hours
# df_supervision_hours['BURDEN_COST_PRCT'].columns.values[0]
# df_supervision_hours.loc[pd.IndexSlice[:, pd.IndexSlice['BURDEN_COST_PRCT',:]]]
df_supervision_hours.style.format('{:.2f}%', subset=pd.IndexSlice[:, pd.IndexSlice['BURDEN_COST_PRCT',:]])
# df_supervision_hours['BURDEN_COST_PRCT']
df_supervision_hours

ROLE TOTAL_HOURS              LABOR_COST  \
MAIN_CONTRACT                              7618       7790         7618   
0                    Craft Subtotal  165,899.25 337,587.33 8,312,039.36   
1              Supervision Subtotal   13,548.00  95,344.75 1,032,247.97   

                            LABOR_COST_PER_HOUR        BURDEN_COST  \
MAIN_CONTRACT          7790                7618  7790         7618   
0             17,672,030.56               50.10 52.35 4,578,274.13   
1              5,822,243.60               76.19 61.07   327,558.08   

                           BURDEN_COST_PRCT      JOB_TOTAL_COST                 
MAIN_CONTRACT         7790             7618 7790           7618           7790  
0             9,531,841.94             0.55 0.54 109,364,282.02 247,860,535.80  
1             2,041,168.11             0.32 0.35 109,364,282.02 247,860,535.80

In [25]:
df_total_hours = df_data.groupby(by=['MAIN_CONTRACT']).agg({'TOTAL_HOURS': 'sum', 'LABOR_COST': 'sum', 'BURDEN_COST': 'sum', 'JOB_TOTAL_COST': 'max'})
df_total_hours = df_total_hours.reset_index()
df_total_hours['ROLE'] = 'Grand Total'
df_total_hours['LABOR_COST_PER_HOUR'] = df_total_hours.LABOR_COST / df_total_hours.TOTAL_HOURS
df_total_hours['BURDEN_COST_PRCT'] = df_total_hours.BURDEN_COST / df_total_hours.LABOR_COST
df_total_hours = df_total_hours.pivot(index='ROLE', columns='MAIN_CONTRACT', values=['TOTAL_HOURS', 'LABOR_COST', 'LABOR_COST_PER_HOUR', 'BURDEN_COST', 'BURDEN_COST_PRCT', 'JOB_TOTAL_COST'])
df_total_hours = df_total_hours.reset_index(names='ROLE')
df_total_hours

ROLE TOTAL_HOURS         LABOR_COST       \
MAIN_CONTRACT                     7484 7618          7484 7618   
0              Grand Total  531,484.48 0.00 26,390,912.24 0.00   

              LABOR_COST_PER_HOUR        BURDEN_COST      BURDEN_COST_PRCT  \
MAIN_CONTRACT                7484 7618          7484 7618             7484   
0                           49.66  NaN 12,782,282.91 0.00             0.48   

                   JOB_TOTAL_COST         
MAIN_CONTRACT 7618           7484   7618  
0              NaN 337,145,526.67 595.18

In [26]:
df_hours = pd.concat([df_craft_hours, df_supervision_hours, df_total_hours])
df_hours.columns.name = 'HOURS'
df_hours = df_hours.reset_index(drop=True).set_index('ROLE')
for _ in df_craft_hours['LABOR_COST'].columns:
    df_hours['TRADE_PRCT_OF_LABOR_TOTAL', _] = df_hours['LABOR_COST', _] / df_hours['LABOR_COST', _].loc[~df_hours.index.get_level_values(0).isin(['Craft Subtotal', 'Grand Total'])].sum()
for _ in df_craft_hours['LABOR_COST'].columns:
    df_hours['TRADE_PRCT_OF_JOB_TOTAL', _] = df_hours['LABOR_COST', _] / df_hours['JOB_TOTAL_COST', _].max()
df_hours

,TOTAL_HOURS,LABOR_COST,LABOR_COST_PER_HOUR,BURDEN_COST,BURDEN_COST_PRCT,JOB_TOTAL_COST,TRADE_PRCT_OF_CRAFT_SUBTOT,TOTAL_HOURS,LABOR_COST,LABOR_COST_PER_HOUR,BURDEN_COST,BURDEN_COST_PRCT,JOB_TOTAL_COST,TRADE_PRCT_OF_LABOR_TOTAL,TRADE_PRCT_OF_JOB_TOTAL
MAIN_CONTRACT,7484,7484,7484,7484,7484,7484,7484,7618,7618,7618,7618,7618,7618,7484,7484
ROLE,,,,,,,,,,,,,,,
Carpenter Composite Crew,"207,480.50","10,040,247.35",48.39,"5,352,725.55",0.53,"337,145,526.67",0.56,NaN,NaN,NaN,NaN,NaN,NaN,0.38,0.03
Cement Mason Composite Crew,"33,024.00","1,571,150.63",47.58,"882,291.94",0.56,"337,145,526.67",0.09,NaN,NaN,NaN,NaN,NaN,NaN,0.06,0.00
Ironworker Composite Crew,"28,849.54","1,396,030.32",48.39,"1,018,232.80",0.73,"337,145,526.67",0.08,NaN,NaN,NaN,NaN,NaN,NaN,0.05,0.00
Laborer Composite Crew,"83,439.50","3,501,890.73",41.97,"1,664,797.77",0.48,"337,145,526.67",0.19,NaN,NaN,NaN,NaN,NaN,NaN,0.13,0.01
Operator Composite Crew,"29,639.36","1,495,963.02",50.47,"891,061.87",0.60,"337,145,526.67",0.08,NaN,NaN,NaN,NaN,NaN,NaN,0.06,0.00
Craft Subtotal,"382,432.90","18,005,072.05",47.08,"9,809,082.54",0.54,"337,145,526.67",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.68,0.05
Supervision Subtotal,"149,055.58","8,409,551.87",56.42,"2,976,047.94",0.35,"337,145,526.67",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.32,0.02
Undefined Subtotal,-4.00,139.60,-34.90,13.61,0.10,"337,145,526.67",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00


In [19]:
df_craft_dist = df_data.loc[(df_data['ROLE_L1'] == 'Craft') & (df_data['MAIN_CONTRACT'] == '7790')].groupby(by=['MAIN_CONTRACT', 'ROLE_L2', 'ROLE_L3']).agg({'TOTAL_HOURS': 'sum'})
df_craft_dist = df_craft_dist.groupby(level=(0,1), group_keys=True).apply(lambda x: x / float(x.iloc[:].sum()))
df_craft_dist.index = df_craft_dist.index.droplevel([0,1])
df_craft_dist = df_craft_dist.rename(columns={'TOTAL_HOURS':'7790'})
df_craft_dist.apply(lambda x: x * 100).style.format('{:.2f}%')

# df_craft_dist = df_data.loc[(df_data['ROLE_L1'] == 'Craft') & (df_data['MAIN_CONTRACT'] == '7618')].groupby(by=['ROLE_L2', 'ROLE_L3']).agg({'TOTAL_HOURS': 'sum'})

# df_craft_dist = df_data.loc[df_data['ROLE_L1'] == 'Craft'].groupby(by=['ROLE_L2', 'ROLE_L3']).agg({'TOTAL_HOURS': 'sum'})
# df_craft_dist = df_craft_dist.groupby(level=(0), group_keys=True).apply(lambda x: 100 * x / float(x.sum()))
# df_craft_dist.index = df_craft_dist.index.droplevel([0])
# df_craft_dist = df_craft_dist.rename(columns={'TOTAL_HOURS':'HOURS_PRCT_OF_TOTAL'})
# df_craft_dist

# pd.concat([df_craft_dist], keys=['7618'], names=['MAIN_CONTRACT'])

C:\Users\SwanS\AppData\Local\Temp\ipykernel_15540\2476756943.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df_craft_dist = df_craft_dist.groupby(level=(0,1), group_keys=True).apply(lambda x: x / float(x.iloc[:].sum()))


In [33]:
frames_dict = {}
idx=0
for job in df_data['MAIN_CONTRACT'].unique():
    df_craft_dist = df_data.loc[(df_data['ROLE_L1'] == 'Craft') & (df_data['MAIN_CONTRACT'] == job)].groupby(by=['MAIN_CONTRACT', 'ROLE_L2', 'ROLE_L3']).agg({'TOTAL_HOURS': 'sum'})
    if not df_craft_dist.dropna().empty:
        df_craft_dist = df_craft_dist.groupby(level=(0,1), group_keys=True).apply(lambda x: 100 * x / float(x.sum()))
        df_craft_dist.index = df_craft_dist.index.droplevel([0,1])
        df_craft_dist = df_craft_dist.rename(columns={'TOTAL_HOURS':job})
        frames_dict[idx] = {}
        frames_dict[idx]['job'] = job
        frames_dict[idx]['df'] = df_craft_dist
        idx+=1
frames_dict

C:\Users\SwanS\AppData\Local\Temp\ipykernel_11284\3559204849.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df_craft_dist = df_craft_dist.groupby(level=(0,1), group_keys=True).apply(lambda x: 100 * x / float(x.sum()))


{0: {'job': '7484',
  'df':                                                      7484
  MAIN_CONTRACT ROLE_L2      ROLE_L3                       
  7484          Carpenter    Apprentice                8.05
                             Foreman                  19.89
                             Journeyman               72.05
                Cement Mason Apprentice                3.28
                             Foreman                  20.67
                             Journeyman               76.05
                Ironworker   Apprentice                9.49
                             Foreman                  30.59
                             Journeyman               57.03
                             Superintendent            2.88
                Laborer      Apprentice               12.36
                             Foreman                  29.68
                             Journeyman               57.96
                Operator     Apprentice               13.99
              

In [27]:
frames_dict = {}
idx=0
for job in df_data['MAIN_CONTRACT'].unique():
    df_craft_dist = df_data.loc[(df_data['ROLE_L1'] == 'Craft') & (df_data['MAIN_CONTRACT'] == job)].groupby(by=['MAIN_CONTRACT', 'ROLE_L2', 'ROLE_L3']).agg({'TOTAL_HOURS': 'sum'})
    df_craft_dist = df_craft_dist.groupby(level=(0,1), group_keys=True).apply(lambda x: 100 * x / float(x.sum()))
    df_craft_dist.index = df_craft_dist.index.droplevel([0,1])
    df_craft_dist = df_craft_dist.rename(columns={'TOTAL_HOURS':job})
    frames_dict[idx] = {}
    frames_dict[idx]['job'] = job
    frames_dict[idx]['df'] = df_craft_dist
    idx+=1
frames_dict

ValueError: Cannot remove 2 levels from an index with 2 levels: at least one level must be left.

In [160]:
# for k,v in frames_dict.items():
#     print(frames_dict[k]['job'])

frames_list = [frames_dict[k]['df'] for k,v in frames_dict.items()]
merge_df = pd.merge(left=frames_list[0], right=frames_list[1], on=['ROLE_L2', 'ROLE_L3']).style.format('{:.2f}%')
for _ in range(len(frames_list[2:])):
    print(frames_dict[1+_]['job'])
    merge_df = pd.merge(left=merge_df, right=frames_list[1+_], on=['ROLE_L2', 'ROLE_L3']).style.format('{:.2f}%')
merge_df

In [127]:
df_craft_dist_2 = df_data.loc[(df_data['ROLE_L1'] == 'Craft') & (df_data['MAIN_CONTRACT'] == '7790')].groupby(by=['MAIN_CONTRACT', 'ROLE_L2', 'ROLE_L3']).agg({'TOTAL_HOURS': 'sum'})
df_craft_dist_2 = df_craft_dist_2.groupby(level=(0,1), group_keys=True).apply(lambda x: 100 * x / float(x.sum()))
df_craft_dist_2.index = df_craft_dist_2.index.droplevel([0,1])
df_craft_dist_2 = df_craft_dist_2.rename(columns={'TOTAL_HOURS':'7790'})
df_craft_dist_2

7790
MAIN_CONTRACT ROLE_L2      ROLE_L3                       
7790          Carpenter    Apprentice               15.09
                           Foreman                  18.13
                           Journeyman               66.77
              Cement Mason Apprentice               14.27
                           Foreman                  17.70
                           Journeyman               68.04
              Ironworker   Apprentice               12.10
                           Foreman                  30.48
                           Journeyman               52.18
                           Superintendent            5.24
              Laborer      Apprentice               15.93
                           Foreman                  22.61
                           Journeyman               61.46
              Operator     Apprentice                1.34
                           Assistant Superintendent  3.32
                           Foreman                   0.57
                           Journeyman               91.56
                           Superintendent            3.21

In [129]:
pd.merge(left=df_craft_dist, right=df_craft_dist_2, on=['ROLE_L2', 'ROLE_L3']).style.format('{:.2f}%')

In [42]:
7735503.95 / 106641754.51

0.07253729072203728

In [ ]:
import os

where_am_i = os.getenv('WHEREAMI')

In [4]:
where_am_i